In [ ]:
'''
Bibliotecas necesarias

! pip install pandas
! pip install requests
! pip install lxml
! pip install bs4
! pip install selenium

Fuente:

Dirección Meteorológica de Chile 2025/02

Catastro de estaciones
https://climatologia.meteochile.gob.cl/application/informacion/buscadorEstaciones

Conjunto por estación

urls_base = [
    "https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_ DiarioTs_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Humedad_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PresionQFF_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Temperatura_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Viento_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Agua24Horas_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Agua6Horas_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_DiarioRR_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PresionQFE_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PuntoRocio_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_TMaxima_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_TMinima_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_IndiceUVB_"
]

'''

In [1]:
import requests
import pandas as pd
import time
import zipfile
import os

In [ ]:
### Lectura de estaciones ### 

# Archivo descargado manualmente de manera previa desde página web y dejado en carpeta "data"
df = pd.read_html("data/CatastrodeEstacionesdelSistemaSACLIM_2025-02-16_21_13.xls")[0] # Requerirá actualización de path

lista_estaciones = df["Codigo Nacional"].values

print("Cantidad de estaciones:", len(lista_estaciones),"\n")
print(lista_estaciones,"\n")

df.head()


Cantidad de estaciones: 46 

[180005 200006 220002 230001 270001 270008 290004 320041 320051 330007
 330019 330020 330021 330030 330031 330066 330077 330111 330112 330113
 330163 340031 360011 360019 360042 370033 380029 390006 400009 410005
 420014 430002 430004 430009 450001 450004 450005 460001 470001 510005
 520006 530005 550001 950001 950002 950003] 



,NÂ°,Codigo Nacional,Codigo WIGOS,Codigo OACI,Nombre,Latitud,Longitud,Altitud (Mts),Propietario
0,0,180005,0-20000-0-85406,SCAR,"Chacalluta, Arica Ap.",-18.35555,-70.34028,51,DMC
1,1,200006,0-20000-0-85418,SCDA,Diego Aracena Iquique Ap.,-20.54917,-70.18111,48,DMC
2,2,220002,0-20000-0-85432,SCCF,"El Loa, Calama Ad.",-22.49806,-68.89250,2321,DMC
3,3,230001,0-20000-0-85442,SCFA,Cerro Moreno Antofagasta Ap.,-23.45361,-70.44528,139,DMC
4,4,270001,0-20000-0-85469,SCIP,Mataveri Isla de Pascua Ap.,-27.15889,-109.43250,69,DMC


In [ ]:
 
# Selenium - Inicio de sesión en Página web y obtención de Cookie de acceso

# SECCIÓN REQUIERE INTERVENCIÓN MANUAL PARA INICIO DE SESIÓN

chromedriver_path = "selenium/chromedriver-win64/chromedriver.exe" # Puede requerir actualización

print("Iniciando selenium")

# Inicio de navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)
driver.get("https://climatologia.meteochile.gob.cl/application/usuario/loginUsuario")

##### NOTA : Aquí uno debe iniciar sesión manualmente. #####
print("Por favor, inicia sesión manualmente en navegador dentro de los próximos 30 segundos")

time.sleep(30)  # Ajustable según sea necesidad para iniciar sesión

# Captura de cookies #
cookies = driver.get_cookies()
print("Cookies capturadas:", cookies)

driver.quit()

Por favor, inicia sesión manualmente en la página que se abrió...
Cookies capturadas: [{'domain': 'climatologia.meteochile.gob.cl', 'httpOnly': False, 'name': 'PHPSESSID', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '9i9o6oj0schochbuddf0k3699p'}]


In [ ]:
### DESCARGA DE VISTAS ###

# Obtención de cookie desde JSON #
cookies_dict = {cookie["name"]: cookie["value"] for cookie in cookies}
print(cookies_dict)


data_folder = "data"
temp_folder = "temp"

# Crear carpeta data en caso de no existir
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# URLS obtenidos 2025/02 desde climatologia.meteochile.gob.cl #
urls_base = [
    "https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_DiarioTs_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Humedad_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PresionQFF_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Temperatura_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Viento_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Agua24Horas_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_Agua6Horas_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_DiarioRR_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PresionQFE_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_PuntoRocio_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_TMaxima_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_TMinima_"
    ,"https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/*estacion*_XXXX_IndiceUVB_"
]

### Bucle de descarga ###

# Para cada URL de la lista #
for url in urls_base:
    # Para cada estación de la lista #
    for n in lista_estaciones:

        # Ajuste de url #
        dinamic_url = url.replace("*estacion*", str(n))
        print(dinamic_url)

        # Ajuste de nombre y path de archivo de salida #
        tema = url.split("_")[-2].rstrip("_/")
        nombre_archivo_zip = f"{temp_folder}\{str(n)}_{tema}.zip"
        
        ## Descarga de vista ##

        # Nota de desarrollo: Son alrededor de 400 vistas las descargadas, de las cuales alrededor de 7 fallan en su extracción
        # Se podría desarrollar a futuro una capacidad para asegurar la solidez de la descarga de datos

        try:
            # Solicitud HTTP a url dinámica utilizando cookie #
            response = requests.get(dinamic_url, cookies=cookies_dict)

            #response.raise_for_status()  # Lanza un error si la descarga falla
            time.sleep(2)

            # Escritura de respuesta binaria en .zip #
            with open(nombre_archivo_zip, "wb") as file:
                file.write(response.content)
            
            print(f"Descargado: {nombre_archivo_zip}\n")

            # Descomprimir el archivo ZIP #
            with zipfile.ZipFile(nombre_archivo_zip, 'r') as zip_ref:
                zip_ref.extractall(data_folder)
                print(f"Archivo descomprimido")

        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {url}: {e} ###############################")
        except zipfile.BadZipFile:
            print(f"Error: El archivo descargado no es un archivo ZIP válido. ##########################")
            

{'PHPSESSID': '9i9o6oj0schochbuddf0k3699p'}
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/180005_XXXX_Humedad_
Descargado: temp\180005_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/200006_XXXX_Humedad_
Descargado: temp\200006_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/220002_XXXX_Humedad_
Descargado: temp\220002_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/230001_XXXX_Humedad_
Descargado: temp\230001_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/270001_XXXX_Humedad_
Descargado: temp\270001_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application/datos/getDatosSaclim/270008_XXXX_Humedad_
Descargado: temp\270008_Humedad.zip
Archivo descomprimido
https://climatologia.meteochile.gob.cl/application

## Identificación de archivos descargados 

In [ ]:

lista_conceptos = [
    "DiarioTs", "Humedad", "PresionQFF", "Temperatura", "Viento",
    "Agua24Horas", "Agua6Horas", "DiarioRR", "PresionQFE",
    "PuntoRocio", "TMaxima", "TMinima", "IndiceUVB"
]

# Ruta de la carpeta "data"
carpeta_data = 'data'

# Diccionario para enlistar los archivos csv por concepto
# {
#  concepto1:[csv1,csv2]; 
#  concepto2:[csv1,csv2];...
# }

archivos_por_concepto = {concepto: [] for concepto in lista_conceptos}

# Clasificación de archivos recibidos por concepto #
# Para cada archivo en la capeta data #
for nombre_archivo in os.listdir(carpeta_data):

    # Verificar concepto por concepto el coincidente con el archivo #
    for concepto in lista_conceptos:
        if nombre_archivo.endswith(f'{concepto}_.csv'):

            # Construir el path completo del archivo y agregar a diccionario #
            path_completo = os.path.join(carpeta_data, nombre_archivo)
            archivos_por_concepto[concepto].append(path_completo)


# Mostrar el diccionario con las listas de archivos por concepto
for concepto, archivos in archivos_por_concepto.items():
    print("{")
    print(f"Concepto: {concepto}")
    print(f"Archivos: {archivos}")
    print(";")
    print("]")

Concepto: DiarioTs
Archivos: ['data\\180005_XXXX_DiarioTs_.csv', 'data\\200006_XXXX_DiarioTs_.csv', 'data\\220002_XXXX_DiarioTs_.csv', 'data\\230001_XXXX_DiarioTs_.csv', 'data\\270001_XXXX_DiarioTs_.csv', 'data\\270008_XXXX_DiarioTs_.csv', 'data\\290004_XXXX_DiarioTs_.csv', 'data\\320041_XXXX_DiarioTs_.csv', 'data\\320051_XXXX_DiarioTs_.csv', 'data\\330007_XXXX_DiarioTs_.csv', 'data\\330019_XXXX_DiarioTs_.csv', 'data\\330020_XXXX_DiarioTs_.csv', 'data\\330021_XXXX_DiarioTs_.csv', 'data\\330030_XXXX_DiarioTs_.csv', 'data\\330031_XXXX_DiarioTs_.csv', 'data\\330066_XXXX_DiarioTs_.csv', 'data\\330077_XXXX_DiarioTs_.csv', 'data\\330111_XXXX_DiarioTs_.csv', 'data\\330112_XXXX_DiarioTs_.csv', 'data\\330113_XXXX_DiarioTs_.csv', 'data\\340031_XXXX_DiarioTs_.csv', 'data\\360011_XXXX_DiarioTs_.csv', 'data\\360019_XXXX_DiarioTs_.csv', 'data\\360042_XXXX_DiarioTs_.csv', 'data\\370033_XXXX_DiarioTs_.csv', 'data\\380029_XXXX_DiarioTs_.csv', 'data\\390006_XXXX_DiarioTs_.csv', 'data\\400009_XXXX_Diario

## Limpieza de datos recibidos

In [ ]:
"""
NOTAS

Tras una inspección visual de los archivos csv resultantes de la descarga:

- Se pudieron identificar varios archivos sin headers, sin la primera fila que tiene sus nombres de columnas
- Se pudo verificar que en todos los archivos la primera fila de headers, estaba separada por , mientras el resto
  del archivo quedó con ; como separador

"""

### Limpieza 1

In [ ]:
# Limpieza 1: Para solucionar fila de headers separa por , en vez de ;
# Nota: Funciona bien gracias a que el separador de decimales es . por lo que no interfiere en caso de que no haya fila de headers

# Función para modificar la primera línea de un archivo y reemplazar , por ;
def modificar_primera_linea(archivo):
    """
    Args
        archivo (path): ruta a uno de los archivos csv
    """
    with open(archivo, 'r') as file:
        lineas = file.readlines()
    
    # Modificar la primera línea reemplazando ',' por ';'
    lineas[0] = lineas[0].replace(',', ';')
    
    with open(archivo, 'w') as file:
        file.writelines(lineas)

# APLICACIÓN EN BUCLE #
for concepto, archivos in archivos_por_concepto.items():
    for archivo in archivos:
        modificar_primera_linea(archivo)

## Limpieza 2

In [ ]:
# EXPLORACION PREVIA #

# Función para verificar la estructura de los archivos de un concepto
def verificar_estructura_concepto(archivos):
    """
    Args
        archivos (lista): Lista de archivos de uno de los conceptos
    """
    estructuras = []

    for archivo in archivos:
        # Cargar el archivo CSV como DataFrame
        df = pd.read_csv(archivo, delimiter=";")
        # Obtener las columnas y tipos de datos
        estructura = {
            'columnas': df.columns.tolist(),
            'tipos_datos': df.dtypes.to_dict()
        }
        # Agregar la estructura a la lista
        estructuras.append(estructura)
    
    # Comparar todas las estructuras con la primera
    primera_estructura = estructuras[0]

    print("Estructura: ", primera_estructura)

    flag_error = 0
    for i, estructura in enumerate(estructuras[1:]):
        if estructura['columnas'] != primera_estructura['columnas']:
            print(f"El archivo {archivos[i+1]} tiene columnas diferentes.")
            print(estructura['columnas'])
            flag_error = 1
        if estructura['tipos_datos'] != primera_estructura['tipos_datos']:
            print(f"El archivo {archivos[i+1]} tiene tipos de datos diferentes.")
            flag_error = 1
    
    return flag_error

# Verificar la estructura para cada concepto
for concepto, archivos in archivos_por_concepto.items():
    if archivos:  # Solo verificar si hay archivos para el concepto
        print(f"Verificando concepto: {concepto}")
        flag_error = verificar_estructura_concepto(archivos)
        if flag_error == 0:
            print(f"Todos los archivos del concepto '{concepto}' tienen la misma estructura.")
        else:
            print(f"¡Advertencia! Los archivos del concepto '{concepto}' no tienen la misma estructura.")
        print("-" * 60)

Verificando concepto: DiarioTs
Estructura:  {'columnas': ['CodigoNacional', 'momento', 'MediaCli_Valor', 'MediaAri_Valor', 'NumDatos_Valor', 'Ts00_Valor', 'Ts12_Valor', 'Maxima_Valor', 'FechaMax_Valor', 'Minima_Valor', 'FechaMin_Valor', 'FechaPro_Valor'], 'tipos_datos': {'CodigoNacional': dtype('int64'), 'momento': dtype('O'), 'MediaCli_Valor': dtype('float64'), 'MediaAri_Valor': dtype('float64'), 'NumDatos_Valor': dtype('float64'), 'Ts00_Valor': dtype('float64'), 'Ts12_Valor': dtype('float64'), 'Maxima_Valor': dtype('float64'), 'FechaMax_Valor': dtype('float64'), 'Minima_Valor': dtype('float64'), 'FechaMin_Valor': dtype('float64'), 'FechaPro_Valor': dtype('float64')}}
Todos los archivos del concepto 'DiarioTs' tienen la misma estructura.
------------------------------------------------------------
Verificando concepto: Humedad
Estructura:  {'columnas': ['CodigoNacional', 'momento', 'HR_Valor'], 'tipos_datos': {'CodigoNacional': dtype('int64'), 'momento': dtype('O'), 'HR_Valor': dtype(

In [ ]:
# Preparación de Limpieza: Agregado de características flag

# El algoritmo de limpieza solo debe ser aplicado a los archivos donde es necesario, para ello primero deben ser identificados con una flag

# Generación de diccionario temporal con flags #

archivos_por_concepto_flag = archivos_por_concepto.copy()

for concepto, archivos in archivos_por_concepto.items():
    archivos_por_concepto_flag[concepto] = [{archivo: {"estructura": 0, "tipos":0}} for archivo in archivos]

# Diccionario para enlistar los archivos csv por concepto agregndo características flag estructura y tipo a cada uno
# {
#  concepto1:[
#   {csv1:{
#       estructura:0;
#       tipos:0
#       }
#   },
#   {csv2:{
#       estructura:0;
#       tipos:0
#       }
#   }
#  ]; 
#  concepto2:[
#   {csv1:{
#       estructura:0;
#       tipos:0
#       }
#   },
#   {csv2:{
#       estructura:0;
#       tipos:0
#       }
#   }
#  ];...
# }


# Verificar la estructura para cada concepto
for concepto, archivos in archivos_por_concepto_flag.items():
    if archivos:  # Solo verificar si hay archivos para el concepto

        print(f"Verificando concepto: {concepto}")

        # Cargar la estructura del primer archivo

        # Nota: Se está asumiendo que el primer archivo tendrá la columna de headers.

        # Nota de desarrollo: Se podría agregar una comprobación de que la primera clave de la primera fila en análisis sea "CodigoNacional", 
        # y sino, pasar al siguiente archivo y vovler a hcer la comprobación para usar de modelo

        primer_archivo = list(archivos[0].keys())[0]
        df_primer = pd.read_csv(primer_archivo, delimiter=";") # Nota: Esto lee todo el archivo, se puede optimizar para leer solo lo primero

        primera_estructura = {
            'columnas': df_primer.columns.tolist(),
            'tipos_datos': df_primer.dtypes.to_dict()
        }

        print("Estructura de referencia: ", primera_estructura)

        # Para cada archivo con sus características
        for dicc in archivos[1:]:
            # Bucle sobre variables/ubicación para modificar en diccionario
            for archivo, dic_atrib in dicc.items():
                
                # Apertura de archivo 
                df = pd.read_csv(archivo, delimiter=";") # Nota: Esto lee todo el archivo, se puede optimizar para leer solo lo primero

                estructura_actual = {
                    'columnas': df.columns.tolist(),
                    'tipos_datos': df.dtypes.to_dict()
                }

                if estructura_actual['columnas'] != primera_estructura['columnas']:
                    print(f"El archivo {archivo} tiene columnas diferentes.")
                    dic_atrib['estructura'] = 1 # Seteo de flag

                if estructura_actual['tipos_datos'] != primera_estructura['tipos_datos']:
                    print(f"El archivo {archivo} tiene tipos de datos diferentes.")
                    dic_atrib['tipos'] = 1 # Seteo de flag


{'DiarioTs': [{'data\\180005_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\200006_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\220002_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\230001_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\270001_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\270008_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\290004_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\320041_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\320051_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330007_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330019_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330020_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330021_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330030_XXXX_DiarioTs_.csv': {'estructura': 0, 'tipos': 0}}, {'data\\330031_XXXX_DiarioTs_.csv'

## Consolidación de archivos

In [ ]:

# Lectura de archivos :

# Abrir los archivos con una opción
    # Condición 0: Se abre con sus header
    # Confición 1: Se abre con los header del primer archivo

# Esto continua asumiendo que el primer archivo tiene sus headers bien.

for concepto, archivos in archivos_por_concepto_flag.items():
    if archivos:  # Solo verificar si hay archivos para el concepto

        print(f"Unificando CSVs: {concepto}")

        # Cargar la estructura del primer archivo
        primer_archivo = list(archivos[0].keys())[0]
        df_primer = pd.read_csv(primer_archivo, delimiter=";")
        columnas_csv = df_primer.columns.tolist()

        print(columnas_csv)

        # lista de dfs para concatenar
        dfs = [df_primer]

        for dicc in archivos[1:]:
            for archivo, dic_atrib in dicc.items():
                if dic_atrib['estructura'] == 0:
                    print(f"{archivo} → Estructura bien")
                    df_temp = pd.read_csv(archivo, delimiter=";")
                elif dic_atrib['estructura'] == 1:
                    print(f"{archivo} → Estructura mal")
                    df_temp = pd.read_csv(archivo, delimiter=";", header=None, names=columnas_csv)
                
                dfs.append(df_temp)

        # Concatenar todos los DataFrames
        df_final = pd.concat(dfs, ignore_index=True)

        # Guardar el DataFrame consolidado
        ruta_salida = f"data/{concepto}_consolidado.csv"
        df_final.to_csv(ruta_salida, index=False, sep=";")
        print(f"Archivo consolidado guardado en: {ruta_salida}")

Unificando CSVs: DiarioTs
['CodigoNacional', 'momento', 'MediaCli_Valor', 'MediaAri_Valor', 'NumDatos_Valor', 'Ts00_Valor', 'Ts12_Valor', 'Maxima_Valor', 'FechaMax_Valor', 'Minima_Valor', 'FechaMin_Valor', 'FechaPro_Valor']
data\200006_XXXX_DiarioTs_.csv → Estructura bien
data\220002_XXXX_DiarioTs_.csv → Estructura bien
data\230001_XXXX_DiarioTs_.csv → Estructura bien
data\270001_XXXX_DiarioTs_.csv → Estructura bien
data\270008_XXXX_DiarioTs_.csv → Estructura bien
data\290004_XXXX_DiarioTs_.csv → Estructura bien
data\320041_XXXX_DiarioTs_.csv → Estructura bien
data\320051_XXXX_DiarioTs_.csv → Estructura bien
data\330007_XXXX_DiarioTs_.csv → Estructura bien
data\330019_XXXX_DiarioTs_.csv → Estructura bien
data\330020_XXXX_DiarioTs_.csv → Estructura bien
data\330021_XXXX_DiarioTs_.csv → Estructura bien
data\330030_XXXX_DiarioTs_.csv → Estructura bien
data\330031_XXXX_DiarioTs_.csv → Estructura bien
data\330066_XXXX_DiarioTs_.csv → Estructura bien
data\330077_XXXX_DiarioTs_.csv → Estructu

In [ ]:
# Revisión de estructuras de csv consolidados generados

for concepto, archivos in archivos_por_concepto_flag.items():
    
    print("Revisando: ", concepto)
    ruta = f"data/{concepto}_consolidado.csv"

    df = pd.read_csv(ruta, delimiter=";")

    print("\n", df.dtypes, "\n")

Revisando:  DiarioTs

 CodigoNacional      int64
momento            object
MediaCli_Valor    float64
MediaAri_Valor    float64
NumDatos_Valor    float64
Ts00_Valor        float64
Ts12_Valor        float64
Maxima_Valor      float64
FechaMax_Valor    float64
Minima_Valor      float64
FechaMin_Valor    float64
FechaPro_Valor    float64
dtype: object 

Revisando:  Humedad

 CodigoNacional      int64
momento            object
HR_Valor          float64
dtype: object 

Revisando:  PresionQFF

 CodigoNacional      int64
momento            object
QFF_Valor         float64
dtype: object 

Revisando:  Temperatura

 CodigoNacional      int64
momento            object
Ts_Valor          float64
dtype: object 

Revisando:  Viento

 CodigoNacional      int64
momento            object
dd_Valor          float64
ff_Valor          float64
VRB_Valor         float64
dtype: object 

Revisando:  Agua24Horas

 CodigoNacional      int64
momento            object
RRR24_Valor       float64
Traza_Valor       float